In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from myusefultools.pyopenfoam import OpenFOAM
from pathlib import Path
import multiprocessing as mp
import os
import itertools

%matplotlib inline
import matplotlib_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('pdf')

In [ ]:
## Check CASES folder exists
cases_folder = Path("CASES")
if not cases_folder.exists():
    os.mkdir(cases_folder)

template_folder = Path("template")

In [ ]:
identifier = f"CASES/casetest"
of = OpenFOAM(
    path_case=identifier, write_to_log = True, path_template=template_folder
)


In [ ]:
def set_boundary_unsatTime(self, t0, tf, h0, hf):
    t0 = t0 * 60
    tf = tf * 60

    ## Set fixedValue but reading from a CSV file
    self.set_value_in_foamDictionary(
        f"{self.latest_time}/h", 
        entry="boundaryField.top",
        value=f"""{{type unsatTime; value {h0}; initialTime {t0}; finalTime {tf}; initialHead {h0}; finalHead {hf}; }}""")
        
## Make that function a bounded method of the OpenFOAM class
setattr(OpenFOAM, set_boundary_unsatTime.__name__, set_boundary_unsatTime)

In [ ]:
from collections import namedtuple
Cycle = namedtuple("Cycle", ["t0", "t1", "t2", "t3", "t4"])


In [ ]:
list_of_runs = [
    Cycle(  0,  41,  88, 102, 173),
    Cycle(173, 251, 287, 311, 411),
    Cycle(411, 461, 499, 521, 617),
    Cycle(617, 671,  708, 732, 822),
    Cycle(822, 880,  917, 940, 1032),
    Cycle(1032, 1089, 1125, 1149, 1243),
    Cycle(1243, 1299, 1340, 1359, 1448),
    Cycle(1448, 1509, 1546, 1569, 1659),
    Cycle(1659, 1720, 1752, 1781, 1867)
    # Cycle(1867, 2015, None, None, None), ## End dry
]

In [ ]:
for i, run in enumerate(list_of_runs):

    of.logger("\n🔥 ======= Dry period ======")
    of.set_endtime(run.t1)
    of.set_boundary_fixedGradient()
    of.set_convergeThreshold(0.250)
    of.cleanup_last_timestep()
    status = of.run_solver()
    of.logger(f"\nSolver ended with return code {status}")
    if status != 0: 
        print(f"Died while drying in cycle {i}", status, of.latest_time)
        break
        
    of.logger("\n🌊 ====== Flood period 1 =====")
    of.set_endtime(run.t2)
    of.set_boundary_unsatTime(run.t1, run.t2, 0, 0.26)
    of.set_convergeThreshold(0.001)
    status = of.run_solver()
    of.logger(f"\nSolver ended with return code {status}")
    if status != 0:
        print(f"Died while flooding in cycle {i}", status, of.latest_time)
        break

    of.logger("\n🌊 ====== Flood period 2 =====")
    of.set_endtime(run.t3)
    of.set_boundary_unsatTime(run.t2, run.t3, 0.26, 0.26)
    status = of.run_solver()
    of.logger(f"\nSolver ended with return code {status}")
    if status != 0:
        print(f"Died while flooding in cycle {i}", status, of.latest_time)
        break
    of.logger("\n🌊 ====== Flood period 3 =====")
    of.set_endtime(run.t4)
    of.set_boundary_unsatTime(run.t3, run.t4, 0.26, 0)
    status = of.run_solver()
    of.logger(f"\nSolver ended with return code {status}")
    if status != 0:
        print(f"Died while flooding in cycle {i}", status, of.latest_time)
        break


In [ ]:
of.foam_to_vtk()
of.boundaryProbes_to_txt()
of.process_boundaryProbes() 

In [ ]:
of.boundaryProbes[0].probes_points

In [ ]:
DIM = 0 # 0 for scalar, 1 for vector
FIELD = "Sw"

fig, ax = plt.subplots()
ax.plot(of.boundaryProbes[DIM].array_data.time, of.boundaryProbes[DIM].array_data[FIELD][0], label="Top")
ax.plot(of.boundaryProbes[DIM].array_data.time, of.boundaryProbes[DIM].array_data[FIELD][1], label="Bottom")
ax.legend()
ax.set_xlabel("Time [s]")
ax.set_ylabel(FIELD)
# ax.set_xlim(0,120_000)
plt.show()

In [ ]:
import json

field = "NO3"

with open("../Rosenzweig_2011/fit_ravid/heatmaps_config.json") as f:
    heatmaps_config = json.load(f)

default_pcolormesh_kwargs = dict(cmap="winter")
pcolormesh_kwargs = heatmaps_config.get(field) or default_pcolormesh_kwargs

of.plot_field_over_time(field, pcolormesh_kwargs);

In [ ]:
sw = of.read_field_all_times("Sw")
porosity = of.read_field_all_times("porosity")
h = of.read_field_all_times("h")

In [ ]:
doc = of.read_field_all_times("DOC")
no3 = of.read_field_all_times("NO3")
nh4 = of.read_field_all_times("NH4")
o2 = of.read_field_all_times("O2")

In [ ]:
fig, ax = plt.subplots()
o2.sel(z=[5.74, 5.24, 4.24, 3.24]).plot.line(x="t", ax=ax)
ax.set_xlabel("Time (s)")
ax.set_ylabel("Sw (-)")
plt.show()

In [ ]:
experimental = pd.read_csv("./experimental-data/.hiddendata/Figure2.csv")
experimental

In [ ]:
depths = [5.74, 5.24, 4.24, 3.24]
colnames = ["WC 25", "WC 75", "WC 175", "WC 275"]
fig, axs = plt.subplots(4,1, figsize=[5,10], sharex=True)

for d,c,ax in zip(depths, colnames, axs):
    theta = (sw*porosity).sel(z=d)
    theta.plot.line(x="t", ax=ax, label="Model")
    ax.set_xlabel("")
    time_experimental = np.arange(len(experimental))*60 # Data acquisition every minute
    ax.plot(time_experimental, experimental[c]/100 , label="Experiment")
    ax.set_ylabel("Sw (-)")
    ax.set_xlim(0, 120_000)
    ax.legend()
ax.set_xlabel("Time (s)")
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=[8,4])
h.sel(z=6.0).plot.line(x="t", ax=ax, label="Model")
ax.plot(time_experimental, experimental["SH"]/100 , label="Experiment")
ax.set_ylim(bottom=0)
ax.set_xlim(0, 120_000)
plt.show()

In [ ]:
depths = [5.74, 5.24, 4.24, 3.24]
colnames = ["OXY 25", "OXY 75", "OXY 175", "OXY 275"]

fig, axs = plt.subplots(5,1, figsize=[5,10], sharex=True)

ax = axs[0]
h.

for d,c,ax in zip(depths, colnames, axs[1:]):
    o2.sel(z=d).plot.line(x="t", ax=ax, label="Model")
    ax.set_xlabel("")
    time_experimental = np.arange(len(experimental))*60 # Data acquisition every minute
    ax.plot(time_experimental, experimental[c]/1000 , label="Experiment")
    ax.set_ylabel("$O_2$ (mg/L)")
    ax.set_xlim(0, 120_000)
    ax.legend()
ax.set_xlabel("Time (s)")
plt.show()